In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

In [3]:
spark = SparkSession \
    .builder \
    .appName("Spark_movie_data_clean") \
    .config("spark.master", "spark://ZHUYANZHAO-HWPC.localdomain:7077") \
    .config("spark.jars.packages", "mysql:mysql-connector-java:8.0.22") \
    .config("spark.executor.cores", "2") \
    .config("spark.executor.num", "1") \
    .getOrCreate()

22/06/18 16:47:12 WARN Utils: Your hostname, ZHUYANZHAO-HWPC resolves to a loopback address: 127.0.1.1; using 172.27.5.87 instead (on interface eth0)
22/06/18 16:47:12 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/george/spark/spark-3.2.1-bin-hadoop3.2/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/george/.ivy2/cache
The jars for the packages stored in: /home/george/.ivy2/jars
mysql#mysql-connector-java added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-fa430396-4227-4737-a313-22189386c0c1;1.0
	confs: [default]
	found mysql#mysql-connector-java;8.0.22 in central
	found com.google.protobuf#protobuf-java;3.11.4 in central
downloading https://repo1.maven.org/maven2/mysql/mysql-connector-java/8.0.22/mysql-connector-java-8.0.22.jar ...
	[SUCCESSFUL ] mysql#mysql-connector-java;8.0.22!mysql-connector-java.jar (3363ms)
downloading https://repo1.maven.org/maven2/com/google/protobuf/protobuf-java/3.11.4/protobuf-java-3.11.4.jar ...
	[SUCCESSFUL ] com.google.protobuf#protobuf-java;3.11.4!protobuf-java.jar(bundle) (1363ms)
:: resolution report :: resolve 8005ms :: artifacts dl 4748ms
	:: modules in use:
	com.google.protobuf#protobuf-java;3.11.4 from central in [default]
	mysql#mysql-connector-java;8.0.22 from central in [d

In [4]:
df_movie = spark.read.option("header","true").csv("./movies_metadata.csv")
df_movie.printSchema()
df_movie.show(5)

root
 |-- adult: string (nullable = true)
 |-- belongs_to_collection: string (nullable = true)
 |-- budget: string (nullable = true)
 |-- genres: string (nullable = true)
 |-- homepage: string (nullable = true)
 |-- id: string (nullable = true)
 |-- imdb_id: string (nullable = true)
 |-- original_language: string (nullable = true)
 |-- original_title: string (nullable = true)
 |-- overview: string (nullable = true)
 |-- popularity: string (nullable = true)
 |-- poster_path: string (nullable = true)
 |-- production_companies: string (nullable = true)
 |-- production_countries: string (nullable = true)
 |-- release_date: string (nullable = true)
 |-- revenue: string (nullable = true)
 |-- runtime: string (nullable = true)
 |-- spoken_languages: string (nullable = true)
 |-- status: string (nullable = true)
 |-- tagline: string (nullable = true)
 |-- title: string (nullable = true)
 |-- video: string (nullable = true)
 |-- vote_average: string (nullable = true)
 |-- vote_count: string (nu

In [5]:
import pyspark.sql.functions as f
df_movie = df_movie.drop("belongs_to_collection").withColumnRenamed("id", "tmdb_id")
df_movie = df_movie.select("*").withColumn("id", f.monotonically_increasing_id() + 1)
df_movie.printSchema()
df_movie.show(5)

root
 |-- adult: string (nullable = true)
 |-- budget: string (nullable = true)
 |-- genres: string (nullable = true)
 |-- homepage: string (nullable = true)
 |-- tmdb_id: string (nullable = true)
 |-- imdb_id: string (nullable = true)
 |-- original_language: string (nullable = true)
 |-- original_title: string (nullable = true)
 |-- overview: string (nullable = true)
 |-- popularity: string (nullable = true)
 |-- poster_path: string (nullable = true)
 |-- production_companies: string (nullable = true)
 |-- production_countries: string (nullable = true)
 |-- release_date: string (nullable = true)
 |-- revenue: string (nullable = true)
 |-- runtime: string (nullable = true)
 |-- spoken_languages: string (nullable = true)
 |-- status: string (nullable = true)
 |-- tagline: string (nullable = true)
 |-- title: string (nullable = true)
 |-- video: string (nullable = true)
 |-- vote_average: string (nullable = true)
 |-- vote_count: string (nullable = true)
 |-- id: long (nullable = false)


In [6]:
df_movie = df_movie.withColumn('imdb_id', f.col('imdb_id').substr(f.lit(4), f.length('imdb_id')))
df_movie.show(5)

+-----+--------+--------------------+--------------------+-------+-------+-----------------+--------------------+--------------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------+--------------------+--------------------+--------+--------------------+-----------------+---+
|adult|  budget|              genres|            homepage|tmdb_id|imdb_id|original_language|      original_title|            overview|popularity|         poster_path|production_companies|production_countries|        release_date|             revenue|             runtime|    spoken_languages|  status|             tagline|               title|   video|        vote_average|       vote_count| id|
+-----+--------+--------------------+--------------------+-------+-------+-----------------+--------------------+--------------------+----------+--------------------+--------------------+--------------------+

In [7]:
df_movie.cache()

DataFrame[adult: string, budget: string, genres: string, homepage: string, tmdb_id: string, imdb_id: string, original_language: string, original_title: string, overview: string, popularity: string, poster_path: string, production_companies: string, production_countries: string, release_date: string, revenue: string, runtime: string, spoken_languages: string, status: string, tagline: string, title: string, video: string, vote_average: string, vote_count: string, id: bigint]

In [17]:
df_movie_genres = df_movie.select(f.col("id").alias("movie_id"), f.col("genres"))
df_movie_genres.printSchema()
df_movie_genres.show(5, truncate=False)

root
 |-- movie_id: long (nullable = false)
 |-- genres: string (nullable = true)

+--------+-------------------------------------------------------------------------------------------------+
|movie_id|genres                                                                                           |
+--------+-------------------------------------------------------------------------------------------------+
|1       |[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}] |
|2       |[{'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'Fantasy'}, {'id': 10751, 'name': 'Family'}]|
|3       |[{'id': 10749, 'name': 'Romance'}, {'id': 35, 'name': 'Comedy'}]                                 |
|4       |[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'name': 'Drama'}, {'id': 10749, 'name': 'Romance'}]    |
|5       |[{'id': 35, 'name': 'Comedy'}]                                                                   |
+--------+-----------------------------------

In [9]:
df_movie_genres.cache()

DataFrame[movie_id: bigint, genres: string]

In [18]:
df_movie_genres2 = df_movie_genres.withColumn("genres", f.from_json(f.col("genres"), "array<string>"))
df_movie_genres2.printSchema()
df_movie_genres2.show(5, truncate=False)

root
 |-- movie_id: long (nullable = false)
 |-- genres: array (nullable = true)
 |    |-- element: string (containsNull = true)

+--------+----------------------------------------------------------------------------------------+
|movie_id|genres                                                                                  |
+--------+----------------------------------------------------------------------------------------+
|1       |[{"id":16,"name":"Animation"}, {"id":35,"name":"Comedy"}, {"id":10751,"name":"Family"}] |
|2       |[{"id":12,"name":"Adventure"}, {"id":14,"name":"Fantasy"}, {"id":10751,"name":"Family"}]|
|3       |[{"id":10749,"name":"Romance"}, {"id":35,"name":"Comedy"}]                              |
|4       |[{"id":35,"name":"Comedy"}, {"id":18,"name":"Drama"}, {"id":10749,"name":"Romance"}]    |
|5       |[{"id":35,"name":"Comedy"}]                                                             |
+--------+------------------------------------------------------------

In [19]:
df_movie_genres2 = df_movie_genres2.withColumn("genres", f.explode("genres"))
df_movie_genres2.show(10, truncate=False)

+--------+-----------------------------+
|movie_id|genres                       |
+--------+-----------------------------+
|1       |{"id":16,"name":"Animation"} |
|1       |{"id":35,"name":"Comedy"}    |
|1       |{"id":10751,"name":"Family"} |
|2       |{"id":12,"name":"Adventure"} |
|2       |{"id":14,"name":"Fantasy"}   |
|2       |{"id":10751,"name":"Family"} |
|3       |{"id":10749,"name":"Romance"}|
|3       |{"id":35,"name":"Comedy"}    |
|4       |{"id":35,"name":"Comedy"}    |
|4       |{"id":18,"name":"Drama"}     |
+--------+-----------------------------+
only showing top 10 rows



In [22]:
import pyspark.sql.types as t
schema = t.StructType(
    [
        t.StructField('id', t.StringType(), True),
        t.StructField('name', t.StringType(), True)
    ]
)
df_movie_genres3 = df_movie_genres2.withColumn('genres', f.from_json('genres', schema))\
.select(f.col('movie_id').cast('int'), f.col('genres.id').cast('int').alias('genre_id'), f.col('genres.name'))
df_movie_genres3.printSchema()
df_movie_genres3.show(10, truncate=False) # Here is the Movie-genre table data

root
 |-- movie_id: integer (nullable = false)
 |-- genre_id: integer (nullable = true)
 |-- name: string (nullable = true)

+--------+--------+---------+
|movie_id|genre_id|name     |
+--------+--------+---------+
|1       |16      |Animation|
|1       |35      |Comedy   |
|1       |10751   |Family   |
|2       |12      |Adventure|
|2       |14      |Fantasy  |
|2       |10751   |Family   |
|3       |10749   |Romance  |
|3       |35      |Comedy   |
|4       |35      |Comedy   |
|4       |18      |Drama    |
+--------+--------+---------+
only showing top 10 rows



In [25]:
#################### Save movie-genre data to MySQL table ####################
df_movie_genres3.select('movie_id', 'genre_id', 'name').write.format("jdbc") \
    .option("url", "jdbc:mysql://localhost:3306/spark_movies") \
    .option("driver", "com.mysql.jdbc.Driver").option("dbtable", "movie_genre") \
    .option("user", "root").option("password", "zhuyz928").save()

In [27]:
df_genres = df_movie_genres3.select(['genre_id', 'name']).distinct()
df_genres.sort(f.col('genre_id')).show(df_genres.count()) # Here is the Genres table data

+--------+--------------------+
|genre_id|                name|
+--------+--------------------+
|      12|           Adventure|
|      14|             Fantasy|
|      14|       Miramax Films|
|      16|           Animation|
|      18|               Drama|
|      27|              Horror|
|      28|              Action|
|      35|              Comedy|
|      36|             History|
|      37|             Western|
|      53|            Thriller|
|      80|               Crime|
|      99|         Documentary|
|     219|The Mirisch Corpo...|
|     878|     Science Fiction|
|    2883|             Aniplex|
|    2957|         Double Play|
|    3279|          Vox3 Films|
|    3287|         Screen Gems|
|    3459|      Serenade Films|
|    6003|     Box Productions|
|    6308|U.S. Office of Wa...|
|    7105|   Mwana Productions|
|    7106|            Sylicone|
|    7759|             GoHands|
|    7760|           BROSTA TV|
|    7761|Mardock Scramble ...|
|    7850|            Sciapode|
|    842

In [28]:
#################### Save genres data to MySQL table ####################
df_genres.select('genre_id', 'name').write.format("jdbc") \
    .option("url", "jdbc:mysql://localhost:3306/spark_movies") \
    .option("driver", "com.mysql.jdbc.Driver").option("dbtable", "genres") \
    .option("user", "root").option("password", "zhuyz928").save()

In [36]:
df_movie_prod_companies = df_movie.select(f.col("id").alias("movie_id"), f.col("production_companies"))
df_movie_prod_companies.printSchema()
df_movie_prod_companies.show(5, truncate=False)

root
 |-- movie_id: long (nullable = false)
 |-- production_companies: string (nullable = true)

+--------+-----------------------------------------------------------------------------------------------------------------------------------+
|movie_id|production_companies                                                                                                               |
+--------+-----------------------------------------------------------------------------------------------------------------------------------+
|1       |[{'name': 'Pixar Animation Studios', 'id': 3}]                                                                                     |
|2       |[{'name': 'TriStar Pictures', 'id': 559}, {'name': 'Teitler Film', 'id': 2550}, {'name': 'Interscope Communications', 'id': 10201}]|
|3       |[{'name': 'Warner Bros.', 'id': 6194}, {'name': 'Lancaster Gate', 'id': 19464}]                                                    |
|4       | determined to find a better way to

In [38]:
df_movie_prod_companies2 = df_movie_prod_companies.withColumn("production_companies", f.from_json(f.col("production_companies"), "array<string>"))
df_movie_prod_companies2.printSchema()
df_movie_prod_companies2.show(5, truncate=False)

root
 |-- movie_id: long (nullable = false)
 |-- production_companies: array (nullable = true)
 |    |-- element: string (containsNull = true)

+--------+--------------------------------------------------------------------------------------------------------------------------+
|movie_id|production_companies                                                                                                      |
+--------+--------------------------------------------------------------------------------------------------------------------------+
|1       |[{"name":"Pixar Animation Studios","id":3}]                                                                               |
|2       |[{"name":"TriStar Pictures","id":559}, {"name":"Teitler Film","id":2550}, {"name":"Interscope Communications","id":10201}]|
|3       |[{"name":"Warner Bros.","id":6194}, {"name":"Lancaster Gate","id":19464}]                                                 |
|4       |null                                      

In [39]:
df_movie_prod_companies2 = df_movie_prod_companies2.na.drop(subset=['production_companies'])
df_movie_prod_companies2.count()

41599

In [40]:
df_movie_prod_companies2 = df_movie_prod_companies2.withColumn("production_companies", f.explode("production_companies"))
df_movie_prod_companies2.show(10, truncate=False)

+--------+-----------------------------------------------+
|movie_id|production_companies                           |
+--------+-----------------------------------------------+
|1       |{"name":"Pixar Animation Studios","id":3}      |
|2       |{"name":"TriStar Pictures","id":559}           |
|2       |{"name":"Teitler Film","id":2550}              |
|2       |{"name":"Interscope Communications","id":10201}|
|3       |{"name":"Warner Bros.","id":6194}              |
|3       |{"name":"Lancaster Gate","id":19464}           |
|5       |{"name":"Sandollar Productions","id":5842}     |
|5       |{"name":"Touchstone Pictures","id":9195}       |
|6       |{"name":"Regency Enterprises","id":508}        |
|6       |{"name":"Forward Pass","id":675}               |
+--------+-----------------------------------------------+
only showing top 10 rows



In [41]:
schema = t.StructType(
    [
        t.StructField('id', t.StringType(), True),
        t.StructField('name', t.StringType(), True)
    ]
)
df_movie_prod_companies3 = df_movie_prod_companies2.withColumn('production_companies', f.from_json('production_companies', schema))\
.select(f.col('movie_id'), f.col('production_companies.id').alias('company_id'), f.col('production_companies.name'))
df_movie_prod_companies3 = df_movie_prod_companies3.na.drop(subset=['company_id', 'name'])
df_movie_prod_companies3.printSchema()
df_movie_prod_companies3.show(10, truncate=False) # Here is the Movie-prod-company table data

root
 |-- movie_id: long (nullable = false)
 |-- company_id: string (nullable = true)
 |-- name: string (nullable = true)

+--------+----------+-------------------------+
|movie_id|company_id|name                     |
+--------+----------+-------------------------+
|1       |3         |Pixar Animation Studios  |
|2       |559       |TriStar Pictures         |
|2       |2550      |Teitler Film             |
|2       |10201     |Interscope Communications|
|3       |6194      |Warner Bros.             |
|3       |19464     |Lancaster Gate           |
|5       |5842      |Sandollar Productions    |
|5       |9195      |Touchstone Pictures      |
|6       |508       |Regency Enterprises      |
|6       |675       |Forward Pass             |
+--------+----------+-------------------------+
only showing top 10 rows



In [50]:
df_companies = df_movie_prod_companies3.select(['company_id', 'name']).distinct()
df_companies.sort(f.col('company_id')).show(10) # Here is the Prod-companies table data
df_companies.count()

+----------+--------------------+
|company_id|                name|
+----------+--------------------+
|         1|           Lucasfilm|
|        10|           Highlight|
|       100|     Avon Production|
|     10000|Compagnia Cinemat...|
|     10001|Andy Warhol Produ...|
|     10002|Filmscope Enterta...|
|     10005|Balthazar Product...|
|      1001|        Solaris Film|
|     10011|     Alta Producción|
|     10012|   Nadie es Perfecto|
+----------+--------------------+
only showing top 10 rows



22093

In [51]:
df_movie_prod_countries = df_movie.select(f.col("id").alias("movie_id"), f.col("production_countries"))
df_movie_prod_countries.printSchema()
df_movie_prod_countries.show(5, truncate=False)

root
 |-- movie_id: long (nullable = false)
 |-- production_countries: string (nullable = true)

+--------+----------------------------------------------------------+
|movie_id|production_countries                                      |
+--------+----------------------------------------------------------+
|1       |[{'iso_3166_1': 'US', 'name': 'United States of America'}]|
|2       |[{'iso_3166_1': 'US', 'name': 'United States of America'}]|
|3       |[{'iso_3166_1': 'US', 'name': 'United States of America'}]|
|4       |3.859495                                                  |
|5       |[{'iso_3166_1': 'US', 'name': 'United States of America'}]|
+--------+----------------------------------------------------------+
only showing top 5 rows



In [52]:
df_movie_prod_countries2 = df_movie_prod_countries.withColumn("production_countries", f.from_json(f.col("production_countries"), "array<string>"))
df_movie_prod_countries2.printSchema()
df_movie_prod_countries2.show(5, truncate=False)

root
 |-- movie_id: long (nullable = false)
 |-- production_countries: array (nullable = true)
 |    |-- element: string (containsNull = true)

+--------+-------------------------------------------------------+
|movie_id|production_countries                                   |
+--------+-------------------------------------------------------+
|1       |[{"iso_3166_1":"US","name":"United States of America"}]|
|2       |[{"iso_3166_1":"US","name":"United States of America"}]|
|3       |[{"iso_3166_1":"US","name":"United States of America"}]|
|4       |null                                                   |
|5       |[{"iso_3166_1":"US","name":"United States of America"}]|
+--------+-------------------------------------------------------+
only showing top 5 rows



In [53]:
df_movie_prod_countries2 = df_movie_prod_countries2.na.drop(subset=['production_countries'])
df_movie_prod_countries2.count()

42472

In [54]:
df_movie_prod_countries2 = df_movie_prod_countries2.withColumn("production_countries", f.explode("production_countries"))
df_movie_prod_countries2.show(10, truncate=False)

+--------+-----------------------------------------------------+
|movie_id|production_countries                                 |
+--------+-----------------------------------------------------+
|1       |{"iso_3166_1":"US","name":"United States of America"}|
|2       |{"iso_3166_1":"US","name":"United States of America"}|
|3       |{"iso_3166_1":"US","name":"United States of America"}|
|5       |{"iso_3166_1":"US","name":"United States of America"}|
|6       |{"iso_3166_1":"US","name":"United States of America"}|
|7       |{"iso_3166_1":"DE","name":"Germany"}                 |
|7       |{"iso_3166_1":"US","name":"United States of America"}|
|8       |{"iso_3166_1":"US","name":"United States of America"}|
|9       |{"iso_3166_1":"US","name":"United States of America"}|
|10      |{"iso_3166_1":"GB","name":"United Kingdom"}          |
+--------+-----------------------------------------------------+
only showing top 10 rows



In [62]:
schema = t.StructType(
    [
        t.StructField('iso_3166_1', t.StringType(), True),
        t.StructField('name', t.StringType(), True)
    ]
)
df_movie_prod_countries3 = df_movie_prod_countries2.withColumn('production_countries', f.from_json('production_countries', schema))\
.select(f.col('movie_id'), f.col('production_countries.iso_3166_1').alias('country_code'), f.col('production_countries.name'))
df_movie_prod_countries3 = df_movie_prod_countries3.na.drop(subset=['country_code', 'name'])
df_movie_prod_countries3.printSchema()
df_movie_prod_countries3.show(10, truncate=False) # Here is the Movie-prod-country table data

root
 |-- movie_id: long (nullable = false)
 |-- country_code: string (nullable = true)
 |-- name: string (nullable = true)

+--------+------------+------------------------+
|movie_id|country_code|name                    |
+--------+------------+------------------------+
|1       |US          |United States of America|
|2       |US          |United States of America|
|3       |US          |United States of America|
|5       |US          |United States of America|
|6       |US          |United States of America|
|7       |DE          |Germany                 |
|7       |US          |United States of America|
|8       |US          |United States of America|
|9       |US          |United States of America|
|10      |GB          |United Kingdom          |
+--------+------------+------------------------+
only showing top 10 rows



In [63]:
df_countries = df_movie_prod_countries3.select(['country_code', 'name']).distinct()
df_countries.sort(f.col('country_code')).show(10) # Here is the Prod-countries table data
df_countries.count()

+------------+--------------------+
|country_code|                name|
+------------+--------------------+
|          AE|United Arab Emirates|
|          AF|         Afghanistan|
|          AL|             Albania|
|          AM|             Armenia|
|          AN|Netherlands Antilles|
|          AO|              Angola|
|          AQ|          Antarctica|
|          AR|           Argentina|
|          AT|             Austria|
|          AU|           Australia|
+------------+--------------------+
only showing top 10 rows



158

In [66]:
df_links = df_movie.select(f.col("id").alias("movie_id"), f.col("tmdb_id"), f.col('imdb_id'))
df_links = df_links.na.drop(subset=['tmdb_id', 'imdb_id'])
df_links.printSchema()
df_links.show(5, truncate=False) # Here is the Links table data
df_links.count()

root
 |-- movie_id: long (nullable = false)
 |-- tmdb_id: string (nullable = true)
 |-- imdb_id: string (nullable = true)

+--------+-------+-------+
|movie_id|tmdb_id|imdb_id|
+--------+-------+-------+
|1       |862    |114709 |
|2       |8844   |113497 |
|3       |15602  |113228 |
|4       |31357  |114885 |
|5       |11862  |113041 |
+--------+-------+-------+
only showing top 5 rows



45447

In [68]:
df_movie = df_movie.drop('genres', 'production_companies', 'production_countries', 'spoken_languages', 'tmdb_id', 'imdb_id')
df_movie.printSchema()
df_movie.show(10, truncate=False) # Here is the moives table data

root
 |-- adult: string (nullable = true)
 |-- budget: string (nullable = true)
 |-- homepage: string (nullable = true)
 |-- original_language: string (nullable = true)
 |-- original_title: string (nullable = true)
 |-- overview: string (nullable = true)
 |-- popularity: string (nullable = true)
 |-- poster_path: string (nullable = true)
 |-- release_date: string (nullable = true)
 |-- revenue: string (nullable = true)
 |-- runtime: string (nullable = true)
 |-- status: string (nullable = true)
 |-- tagline: string (nullable = true)
 |-- title: string (nullable = true)
 |-- video: string (nullable = true)
 |-- vote_average: string (nullable = true)
 |-- vote_count: string (nullable = true)
 |-- id: long (nullable = false)

+-----+--------+--------------------------------------------+-----------------+---------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------

In [69]:
df_ratings = spark.read.option("header","true").csv("./ratings.csv")
df_ratings = df_ratings.drop('userId').withColumnRenamed('movieId', 'movie_id').na.drop(subset=['movie_id', 'rating'])
df_ratings.printSchema()
df_ratings.show(10, truncate=False) # Here is the Ratings table data

root
 |-- movie_id: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- timestamp: string (nullable = true)

+--------+------+----------+
|movie_id|rating|timestamp |
+--------+------+----------+
|110     |1.0   |1425941529|
|147     |4.5   |1425942435|
|858     |5.0   |1425941523|
|1221    |5.0   |1425941546|
|1246    |5.0   |1425941556|
|1968    |4.0   |1425942148|
|2762    |4.5   |1425941300|
|2918    |5.0   |1425941593|
|2959    |4.0   |1425941601|
|4226    |4.0   |1425942228|
+--------+------+----------+
only showing top 10 rows



22/06/13 23:41:22 ERROR TaskSchedulerImpl: Lost executor 0 on 172.27.7.53: Remote RPC client disassociated. Likely due to containers exceeding thresholds, or network issues. Check driver logs for WARN messages.
22/06/13 23:41:22 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_46_1 !
22/06/13 23:41:22 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_46_0 !
22/06/13 23:41:22 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_57_0 !
22/06/13 23:41:22 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_116_1 !
22/06/13 23:41:22 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_116_0 !
22/06/13 23:41:22 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_57_1 !
22/06/13 23:41:48 WARN StandaloneAppClient$ClientEndpoint: Connection to ZHUYANZHAO-HWPC.localdomain:7077 failed; waiting for master to reconnect...
22/06/13 23:41:48 WARN StandaloneSchedulerBackend: Disconnected from Spark clust